# Watson Assistant Performance Metrics [Florida Department of Health ELI]

This notebook performs analytics on the log records of Watson Assistant. A variety of key business metrics are calculated and saved in CSV files. Optionally, this CSV data can be used to be embed into a Cognos Dashboard in Watson Studio. 

Logs can also be exported to spreadsheet.

## Housekeeping
This section will import libraries and dependencies for this notebook. 

**Action Required:** 
- Update the `project_id` and `project_access_token` in order to access your data assets.
- Upload `getAllLogs.py` and `extractConversations.py` into your project's assets. They can be found at https://github.com/cognitive-catalyst/WA-Testing-Tool/tree/master/log_analytics

### Insert Project Token

In [1]:
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='ec5e7dab-fb70-4af6-96b5-e79b44b0f945', project_access_token='p-ddf42113160f0687b303b59417f242c48a9e3d96')
pc = project.project_context

### Import Dependencies

In [2]:
# Import dependencies. Ensure these are loaded into your Studio assets.
fobj = open("getAllLogs.py", "wb")
fobj.write(project.get_file("getAllLogs.py").read()) 
fobj.close()

fobj = open("extractConversations.py", "wb")
fobj.write(project.get_file("extractConversations.py").read()) 
fobj.close()

fobj = open("intent_heatmap.py", "wb")
fobj.write(project.get_file("intent_heatmap.py").read()) 
fobj.close()

In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Import Needed Modules

In [4]:
import warnings
warnings.simplefilter("ignore")
!pip install ibm-watson
import json
import calendar
import pandas as pd
import getAllLogs
import extractConversations
import string

### Define Important Variables
This cell will house all the important variables to be used throughout the notebook.

In [5]:
    # Define the customer name. This prefix will be used for saving CSV & JSON files.
custName = 'BON_weekly'

    # Define the dates that analytics are to be ran between in "yyyy-mm-dd" format. Note: You will not be able to pull any results past 30 of the current date.
start_timestamp = "2020-12-21"
end_timestamp = "2020-12-27"

    # assistant_id can be found under the Settings -> API details for each assistant.
assistant_id = "88bbcf5b-bad3-41c5-bd47-b3b8c3fc5be4"

    # API, URL, and workspace ID are extractable from "View API Details page"
iam_apikey="s6EfQILQuGhIfaOwRfMroYKlkFmEcwrFSFvX3BJxfF-M"
url="https://gateway.watsonplatform.net/assistant/api/"
workspace_id=None # Update or set to None without quotes. If assistant_id is has a value, set this to None.
version="2018-09-20"

    # Below are important nodes that need to be identified in order to calulate metrics. These can be found in the JSON of each Watson Assistant Skill.

    # "Anything Else" nodes for calculating coverage rate.
general_negative_feedback = "node_7_1580251456692"
anything_else_node = "node_2_1467831978407"
wds_search_node = "node_5_1580945934910"

    # "Escalation" nodes for calculating escalation metrics
escalation_node = "node_3_1585767656658"
liveagent_node = "node_2_1586552103721"
outofhours_node = "node_1_1594224279859"

    # "Chat Queue" nodes for calculating chat queue metrics
board_chat_node = "node_1_1586548298557"
pub_rec_chat_node = "node_2_1586532252878"
bgs_chat_node = "node_3_1586532252878"
lss_chat_node = "node_4_1586532252878"
cc_chat_node = "node_5_1586532252878"

### Retrieve logs from the Watson Assistant Instance

Conversation logs are retrieved from the watson assistant instance and are saved in JSON format.The option to save the raw json is available at the bottom of the cell if needed 

In [6]:

    # Filter API is described at: https://cloud.ibm.com/docs/assistant?topic=assistant-filter-reference#filter-reference
log_filter="language::en,response_timestamp>="+ start_timestamp + ",response_timestamp<=" + end_timestamp + ",request.context.system.assistant_id::" + assistant_id  # If using this, uncomment and replace <<assistant_id>>

    #Change the number of logs retrieved, default settings will return 100,000 logs (200 pages of 500)
page_size_limit=500
page_num_limit=20000

    # Retrieves logs and saves them in JSON format
rawLogsJson = getAllLogs.getLogs(iam_apikey, url, workspace_id, log_filter, page_size_limit, page_num_limit, version)
rawLogsPath= custName + "_logs.json"


    # Uncomment one of the two lines below if you want to save a copy of the created JSON.
#getAllLogs.writeLogs(rawLogsJson, rawLogsPath) # Saves the logs locally
#project.save_data(file_name = rawLogsPath,data = json.dumps(rawLogsJson),overwrite=True); # Saves the logs in Studio/COS
    
    # Print Statements
#print('\nSaved log data to {}'.format(rawLogsPath))

Fetched 1 log pages
Fetched 2 log pages
Fetched 3 log pages
Fetched 4 log pages
Fetched 5 log pages
Fetched 6 log pages
Fetched 7 log pages


### Format Logs 

The logs are extracted and formated from the raw JSON data and saved into a Dataframe.

In [7]:
    # Define the conversation correlation field name for your Watson Assistant records.
    # Provide the field name as it appears in the log payload (default is 'response.context.conversation_id')
    # For a single-skill assistant use 'response.context.conversation_id'
    # For a Voice Gateway/Voice Agent assistant use 'request.context.vgwSessionID'
    # For a multi-skill assistant you will need to provide your own key
primaryLogKey = "response.context.conversation_id"

    # Name of the correlating key as it appears in the data frame columns (remove 'response.context.')
conversationKey='conversation_id'

    # Optionally provide a comma-separated list of custom fields you want to extract, in addition to the default fields
customFieldNames = "queue"

    #Formats and saves JSON data into a Pandas DataFrame
allLogsDF = extractConversations.extractConversationData(rawLogsJson, primaryLogKey, customFieldNames)
conversationsGroup = allLogsDF.groupby(conversationKey,as_index=False)

    # Print Statements
print("Total log events:",len(allLogsDF))
print(allLogsDF.columns)
allLogsDF.head()


Total log events: 3081
Index(['conversation_id', 'dialog_turn_counter', 'request_timestamp',
       'response_timestamp', 'input.text', 'output.text', 'intent',
       'intent_confidence', 'nodes_visited', 'branch_exited_reason', 'queue',
       'entities', 'prev_nodes_visited', 'conversation_start', 'message_start',
       'message_end', 'dialog_turn_number', 'conversation_length'],
      dtype='object')


,conversation_id,dialog_turn_counter,request_timestamp,response_timestamp,input.text,output.text,intent,intent_confidence,nodes_visited,branch_exited_reason,queue,entities,prev_nodes_visited,conversation_start,message_start,message_end,dialog_turn_number,conversation_length
0,002674bd-90c3-4949-96b3-fbe68cb657f7,1.0,2020-12-22 17:59:24.149000+00:00,2020-12-22 17:59:24.170000+00:00,Start Conversation,"Hello, I am ELI, a virtual assistant chatbot (...",Bot_Control_Start_Over,0.741890,"(Opening,)",completed,,,,2020-12-22 17:59:24.149000+00:00,00:00:00,00:00:00,1,1
1,002c357b-fe98-46b1-93aa-f4c13447a192,1.0,2020-12-22 19:57:00.605000+00:00,2020-12-22 19:57:00.628000+00:00,Start Conversation,"Hello, I am ELI, a virtual assistant chatbot (...",Bot_Control_Start_Over,0.741890,"(Opening,)",completed,,,,2020-12-22 19:57:00.605000+00:00,00:00:00,00:00:00,1,1
2,004f5818-8b1f-4447-b9e4-6df01beae299,1.0,2020-12-22 15:55:23.277000+00:00,2020-12-22 15:55:23.306000+00:00,Start Conversation,"Hello, I am ELI, a virtual assistant chatbot (...",Bot_Control_Start_Over,0.741890,"(Opening,)",completed,,,,2020-12-22 15:55:23.277000+00:00,00:00:00,00:00:00,1,1
3,0051868f-c196-4ffb-9a2c-d7e9f692d512,1.0,2020-12-27 14:23:57.477000+00:00,2020-12-27 14:23:57.509000+00:00,Start Conversation,"Hello, I am ELI, a virtual assistant chatbot (...",Bot_Control_Start_Over,0.741890,"(Opening,)",completed,,,,2020-12-27 14:23:57.477000+00:00,00:00:00,00:00:00,1,2
4,0051868f-c196-4ffb-9a2c-d7e9f692d512,2.0,2020-12-27 14:24:04.725000+00:00,2020-12-27 14:24:04.761000+00:00,END_SESSION,"I'm sorry, I'm not understanding. Can you reph...",General_Ending,0.341786,"(node_1_1606141961460, node_2_1467831978407)",completed,,,"(Opening,)",2020-12-27 14:23:57.477000+00:00,00:00:00.032000,00:00:07.248000,2,2


### Save All Log to CSV

In [8]:
     # Uncomment one of the two lines below if you want to save a copy of the created All Log Dataframe to a CSV.
#allLogsDF.to_csv(custName + "_logs.csv",index=False)# This saves if running code locally. Comment out for Studio. 
project.save_data(file_name = custName + "_logs.csv",data = allLogsDF.to_csv(index=False),overwrite=True) # This saves in COS. Comment out if running locally

    # Print Statements
print('Saving metrics to {}'.format(custName+ "_logs.csv"))

Saving metrics to BON_weekly_logs.csv


# Key Metrics 

The next few cells will calculate key metrics needed for analysis and save them to a CSV file that will be used to populate the Cognos Dashboard.<br><br>
These metrics include:
- **Log Search Timeframe** The selected start and end date for the log search.
- **Total Number of Messages** The number of individual messages presented in the log search.
- **Total Number of Conversations** The number of individual conversations presented in the log search.
- **Number of Covered Messages** The number of messages that watson assistant could give a curated response for.
- **Coverage Rate** The percentage of covered messages.
- **Number of Escalations** The number of time that a possible escalation to a live agent took place.
- **Escalation Rate** The percentage of escalated occurrences in comparison to the overall number of messages.
- **Number of Liveagent Transfers** The number of escalations that were sent to a live agent chat.
- **Liveagent Transfer Rate** The percentage of live agent chat occurrences in comparison to the overall number of messages.
- **Number of "Out of Hours" Transfers** The number of attempt to speak to a live agent out side of operating hours.
- **"Out of Hours" Transfer Rate** The percentage of "Out of Hours" occurrences in comparison to the overall number of messages.
- **Number of Abandoned Transfers** The number of time a user accepted to speak to an agent but did not select a queue to trigger a live agent chat.
- **Abandoned Transfer Rate** The percentage of abandoned transfer occurrences in comparison to the overall number of messages.

## Timeframe, Message, and Conversation Metrics
- The selected timeframe is added to the key metrics.
- The log is filtered for any empty inputs
- The number of messages and conversations are added to the key metrics

In [9]:
    # This dict will contain the key metric data that will be later saved into a Pandas Dataframe.
metrics_dict = {}

    # Save dates defined to dict.
metrics_dict['start_timestamp'] = [start_timestamp]
metrics_dict['end_timestamp'] = [end_timestamp]

    # Filter out blank inputs in log events and saves them to a Pandas Dataframe.
filteredLogsDF = allLogsDF[allLogsDF['input.text'] != ""]

    # These should match the count in the Watson Assistant Analytics tooling.
totalMessages= len(allLogsDF)
totalConvs = len(allLogsDF[conversationKey].unique())

    # If any abandoned conversations are found this be different from the count above.
filteredMessages = len(filteredLogsDF)
filteredConvs = len(filteredLogsDF[conversationKey].unique())
 
    # Save filterd message and conversation counts to dict.
metrics_dict['totalMessages'] = [filteredMessages] 
metrics_dict['totalConvs'] = [filteredConvs] 

    # Print Statements
print("Total Messages/Total Conversations Saved to metrics_dict")
print("Total messages:", totalMessages)
print("Total conversations:", totalConvs) 
print("Abandoned conversations:", totalConvs - filteredConvs)

Total Messages/Total Conversations Saved to metrics_dict
Total messages: 3081
Total conversations: 733
Abandoned conversations: 0


## Coverage 
- The number of covered messages and coverage rate are added to the key metrics.

In [10]:
    # Saves node_id for anything_else and other uncovered nodes to a list.
anything_else_nodes = [anything_else_node,wds_search_node,general_negative_feedback]

    # Nested Loop finds the number all instances of uncovered messages.
for row in filteredLogsDF.itertuples():
     nodes = row.nodes_visited
     for node in nodes:
         if node in anything_else_nodes:
             anything_else_nodes.append(row.Index)
            
    # Saves uncoverd messages to a Pandas DataFrame
uncoveredDF = filteredLogsDF[filteredLogsDF.index.isin(anything_else_nodes)]

    # Finds the coverage rate and number of covered messages from the created dataframes.
coverageMetric = 1-len(uncoveredDF)/filteredMessages
totalCoveredMessages = totalMessages - len(uncoveredDF)

    # Saves percentage and counts to dict.
metrics_dict['coverage_count'] = totalMessages - len(uncoveredDF)
metrics_dict['coverage_pcent'] = 1-len(uncoveredDF)/filteredMessages

    # Print Statements
print("Number of Covered Messages/Coverage Percentage Saved to metrics_dict")
print("Covered messages:",totalCoveredMessages)
print("Coverage percentage:",'{:.1%}'.format(coverageMetric))

Number of Covered Messages/Coverage Percentage Saved to metrics_dict
Covered messages: 2786
Coverage percentage: 90.4%


## Optional: Save Uncovered Messages
- Saves a CSV file of uncovered messages that can be reviewed later WA training improvments.

In [11]:
    #Reformates dataframe for CSV export
#uncoveredDF = uncoveredDF[['input.text','output.text','intent','intent_confidence']]

project.save_data(file_name = custName + "_uncovered_msgs.csv",data = uncoveredDF.to_csv(index=False),overwrite=True); # This saves in COS. Comment out if running locally
    # Uncomment one of the two lines below if you want to save a CSV copy of uncoered messages found.
#uncoveredDF.to_csv(custName + "_uncovered_msgs.csv",index=False, header=['Utterance','Response','Intent','Confidence']) # This saves if running code locally. Comment out for Studio.
#project.save_data(file_name = custName + "_uncovered_msgs.csv",data = uncoveredDF.to_csv(index=False,header=['Utterance','Response','Intent','Confidence']),overwrite=True); # This saves in COS. Comment out if running locally

    # Print Statements
uncoveredDF.head(10).sort_values('intent_confidence',ascending=False)
print('\nSaved', len(uncoveredDF), 'messages to:', custName + "_uncovered_msgs.csv")


Saved 295 messages to: BON_weekly_uncovered_msgs.csv


## Escalations
- Number of escalations and escalation rate are added to key metrics.

In [12]:
    # Add in the node id for any nodes that are like an escalation.
node_visits_escalated = allLogsDF[[escalation_node in x for x in allLogsDF['nodes_visited']]]

    # Calculates the number of escalations and the escalation percentage
totalEscalations = len(node_visits_escalated)
escalationMetric = len(node_visits_escalated)/filteredConvs

    # Saves escalation count and escalation percentage to dict
metrics_dict['escalation_count'] = [totalEscalations]
metrics_dict['escalation_pcent'] = [escalationMetric]

    # Print Statements
print("Escalation Total/Escalation Percentage Saved to metrics_dict")
print("Total Escalations:",totalEscalations)
print("Escalation Percentage:",'{:.1%}'.format(escalationMetric))

Escalation Total/Escalation Percentage Saved to metrics_dict
Total Escalations: 115
Escalation Percentage: 15.7%


## Liveagent Transfers
- Number of liveagent transfers and liveagent transfer rate are added to key metrics.

In [13]:
    # This is the node id for transfer to live agent
node_visits_txfr_live_agent = allLogsDF[[liveagent_node in x for x in allLogsDF['nodes_visited']]]

    # Calculates the number of liveagent transfers and the liveagent transfers percentage.
totalLiveagentTansfers = len(node_visits_txfr_live_agent)
liveagentMetric = len(node_visits_txfr_live_agent)/filteredConvs

    # Saves liveagent transfers count and liveagent transfers percentage to dict.
metrics_dict['liveagent_count'] = [totalLiveagentTansfers]
metrics_dict['liveagent_pcent'] = [liveagentMetric]

    # Print Statements
print("Total Liveagent Transfers/Liveagent Transfer Percentage Saved to metrics_dict")
print("Total Liveagent Transfers:",totalLiveagentTansfers)
print("Liveagent Transfer Percentage",'{:.1%}'.format(liveagentMetric))

Total Liveagent Transfers/Liveagent Transfer Percentage Saved to metrics_dict
Total Liveagent Transfers: 73
Liveagent Transfer Percentage 10.0%


## "Out of Hours" Transfers
- Number of "Out of Hours" Transfers and "Out of Hour" Transfer Rate are added to key metrics.

In [14]:
    # This is the node id for transfer to live agent Out of Hours
node_visits_outofhours = allLogsDF[[outofhours_node in x for x in allLogsDF['nodes_visited']]]

    # Calculates the number of liveagent transfers Out of Hours and the liveagent transfers Our of hours percentage.
totalOutOfHours = len(node_visits_outofhours)
outofhoursMetric = len(node_visits_outofhours)/filteredConvs

    # Saves liveagent transfers Out of hours count and liveagent transfers out of hours percentage to dict.
metrics_dict['outofhours_count'] = [totalOutOfHours]
metrics_dict['outofhours_pcent'] = [outofhoursMetric]

    # Print Statements
print("Total Liveagent Transfers OoH/Liveagent Transfer OoH Percentage Saved to metrics_dict")
print("Total Out of Hours Transfers:",totalOutOfHours)
print("Out of Hours Transfers Percentage",'{:.1%}'.format(outofhoursMetric))

Total Liveagent Transfers OoH/Liveagent Transfer OoH Percentage Saved to metrics_dict
Total Out of Hours Transfers: 24
Out of Hours Transfers Percentage 3.3%


## Abandoned Transfers
- Number of Abandonded Transfers and Abandoned Transfer Rate are added to key metrics.

In [15]:
    # Calculates the number of abandoned transfers and the abandoned transfers percentage.
totalAbandonedTransfers = totalEscalations - totalLiveagentTansfers - totalOutOfHours
abandonedTransfersMetric = totalAbandonedTransfers/filteredConvs

    # Saves abandoned transfers count and abandoned transfers percentage to dict.
metrics_dict['abandoned_count'] = [totalAbandonedTransfers]
metrics_dict['abandoned_pcent'] = [abandonedTransfersMetric]

    # Print Statements
print ("Total Abandoned Transfers/Abandoned Transfer Percentage Saved to metrics_dict")
print("Total Abondoned Transfers:",totalAbandonedTransfers)
print("Abondoned Transfers Percentage",'{:.1%}'.format(abandonedTransfersMetric))

Total Abandoned Transfers/Abandoned Transfer Percentage Saved to metrics_dict
Total Abondoned Transfers: 18
Abondoned Transfers Percentage 2.5%


## Save Key Metrics
- Saves a CSV file of key metrics for review or in conjunction with Cognos Dashboard.

In [16]:
    # Saves metrics collected from dict to Pandas Dataframe.
metricsDF = pd.DataFrame(metrics_dict)

    # Uncomment one of the two lines below if you want to save a copy of the key metrics as a CSV.
#metricsDF.to_csv(custName + "_KeyMetrics.csv",index=False) # This saves if running code locally. Comment out for Studio. 
project.save_data(file_name = custName + "_KeyMetrics.csv",data = metricsDF.to_csv(index=False),overwrite=True); # This saves in COS. Comment out if running locally

    # Print Statements
print('Saving key metrics to {}'.format(custName+ "_KeyMetrics.csv"))

Saving key metrics to BON_weekly_KeyMetrics.csv


# Daily Conversations and Messages Metric

The following few cells will calculate the number of conversations and messages for each day of the week and save them to a CSV file.

## Daily Messages

In [17]:
    # Copies all log dataframe to new dataframe 
time = allLogsDF.copy()

    # Adds a new column that identifies the day of the week the message took place based on the timestamp
time['weekday'] = time['request_timestamp'].apply(lambda x: x.weekday())

    # Creates a new Pandas dataframe that will hold the message count per day of the week.
weekday_freq = pd.DataFrame( columns=["count"], index=[0,1,2,3,4,5,6])

    # Nested loop calculates the number of times a message falls on a curtain day of the week and adds it to weekday_freq dataframe.
for i in weekday_freq.index:
     count = 0
     for j in time.index:
         if i == time.loc[j, 'weekday']:
             count += 1
     weekday_freq.loc[i, "count"] = count
    
    # Adds new column to dataframe that stamps the weekday name based on the index number.
weekday_freq['Weekdays'] = calendar.day_name

    # Print Statements
weekday_freq

,count,Weekdays
0,743,Monday
1,793,Tuesday
2,776,Wednesday
3,309,Thursday
4,93,Friday
5,207,Saturday
6,160,Sunday


## Daily Conversations

In [18]:
    # Creates a new Pandas dataframe that will hold the conversation count per day of the week.
weekday_conversations = pd.DataFrame(columns= ["conversation_count"], index = [0,1,2,3,4,5,6])

    # Nested loop calculates the number of times a message falls on a certian day of the week and adds it to weekday_conversations dataframe.
for i in weekday_conversations.index:
     id_list = []
     for j in time.index:
         if (i == time.loc[j,"weekday"]) and (time.loc[j,"conversation_id"] not in id_list):
             id_list.append(time.loc[j, "conversation_id"])
     weekday_conversations.loc [i, "conversation_count"] = (len(id_list))
    
    # Adds new column to dataframe that stamps the weekday name based on the index number.
weekday_conversations['Weekdays'] = calendar.day_name

    # Print Statements
weekday_conversations

,conversation_count,Weekdays
0,177,Monday
1,191,Tuesday
2,157,Wednesday
3,83,Thursday
4,27,Friday
5,50,Saturday
6,51,Sunday


## Merge Daily Messages/Conversations

In [19]:
    # Merges weekday dataframes created above.
weekday_count = pd.DataFrame.from_dict({"Day of Week": [0,1,2,3,4,5,6],
                                       "Daily Conversations": weekday_conversations['conversation_count'],
                                       "Daily Messages": weekday_freq['count'],
                                       "Weekdays": weekday_conversations['Weekdays']})
    # Print Statements
weekday_count

,Day of Week,Daily Conversations,Daily Messages,Weekdays
0,0,177,743,Monday
1,1,191,793,Tuesday
2,2,157,776,Wednesday
3,3,83,309,Thursday
4,4,27,93,Friday
5,5,50,207,Saturday
6,6,51,160,Sunday


## Save Daily Conversations and Messages Metric
- Saves a CSV file containing the Daily Conversations and Messages Metric.

In [20]:
    # Uncomment one of the two lines below if you want to save a copy of the weekday conversations and messages count dataframe as a CSV.
#weekday_count.to_csv(custName + "_Weekday_Conversation_Messages.csv",index=False) # This saves if running code locally. Comment out for Studio. 
project.save_data(file_name=custName+"_Weekday_Conversation_Messages.csv", data=weekday_count.to_csv(index=False), overwrite=True)# This saves in COS. Comment out if running locally.

    # Print Statements
print('Saving key metrics to {}'.format(custName+ "_Weekday_Conversation_Messages.csv"))

Saving key metrics to BON_weekly_Weekday_Conversation_Messages.csv


# Chat Queue Metric
- The next couple of cells will count the number times each chat agent queue was triggered and saves the results in a CSV file.

## Queue Count
- The log is searched for each indevidual triggered queue and tallies them.

In [21]:
    # Define the node_id for transfer to live agent nodes: node_1=Board call center, node_2=public records, node_3=background screening, node_4=licensure support, node_5=customer contact center
queue_nodes = [board_chat_node,pub_rec_chat_node,bgs_chat_node,lss_chat_node,cc_chat_node] 

    # Creats a new dataframe with queue names and counts (starting at 0).
queueCountDF = pd.DataFrame({'Queue Count':[0,0,0,0,0],
                             'Webchat Queues' :['Board','Public Records', 'Background Screening', 'Licensure Support', "Contact Center"]})
    # Calculates the number of times a specific queue
for i in filteredLogsDF.itertuples():
     nodes = i.nodes_visited
     for j in nodes:
         if j in queue_nodes:
             if j == board_chat_node:
                 queueCountDF.loc[0,'Queue Count'] = queueCountDF.loc[0,'Queue Count'] + 1
             elif j == pub_rec_chat_node:
                 queueCountDF.loc[1,'Queue Count'] = queueCountDF.loc[1,'Queue Count'] + 1
             elif j == bgs_chat_node:
                 queueCountDF.loc[2,'Queue Count'] = queueCountDF.loc[2,'Queue Count'] + 1
             elif j == lss_chat_node:
                 queueCountDF.loc[3,'Queue Count'] = queueCountDF.loc[3,'Queue Count'] + 1
             elif j == cc_chat_node:
                 queueCountDF.loc[4,'Queue Count'] = queueCountDF.loc[4,'Queue Count'] + 1
                
    # Print Statements
queueCountDF

,Queue Count,Webchat Queues
0,23,Board
1,0,Public Records
2,10,Background Screening
3,14,Licensure Support
4,17,Contact Center


## Save Chat Queue Metric
- Saves a CSV file of Chat queue metric.

In [22]:
    # Uncomment one of the two lines below if you want to save a copy of the queue count dataframe as a CSV.
#queueCountDF.to_csv(custName + "_QueueCount.csv",index=False )# This saves if running code locally. Comment out for Studio.
project.save_data(file_name = custName + "_QueueCount.csv",data = queueCountDF.to_csv(index=False),overwrite=True) # This saves in COS. Comment out if running locally

    # Print Statements
print('Saving key metrics to {}'.format(custName+ "_QueueCount.csv"))

Saving key metrics to BON_weekly_QueueCount.csv
